In [1]:
import numpy as np

def validar_fuerza_ciclo(ticker, flujo_hft, volumen_hft):
    """
    Toma la decisión final basada en el CVD RELATIVO (Eficiencia).
    volumen_hft: Debe ser un deque similar al flujo_hft pero con (compra + venta).
    """
    estado = ESTADO_MERCADO[ticker]
    
    if not estado["en_referencia"]:
        return "ESPERAR"

    # 1. Calculamos la Eficiencia (CVD Relativo) de los últimos 100 ticks
    # Usamos 100 para tener una muestra estadística real y evitar el ruido de los últimos 10.
    ventana = 100
    if len(flujo_hft) < ventana: return "ESPERAR"

    cvd_reciente = np.sum(list(flujo_hft)[-ventana:])
    vol_reciente = np.sum(list(volumen_hft)[-ventana:])
    
    # EFICIENCIA: ¿Qué porcentaje del volumen es delta real?
    eficiencia = (cvd_reciente / vol_reciente * 100) if vol_reciente > 0 else 0

    # --- ESCENARIO A: BUSCANDO ENTRADA (COMPRA) ---
    if not estado["posicion_activa"]:
        # Solo compramos en bandas inferiores si la dominancia de compra > 12%
        if estado["fase_ciclo"] in ["inf2", "inf1"] and eficiencia > 12.0:
            print(f"🔥 COMPRA VALIDADA | Eficiencia: {eficiencia:.2f}%")
            return "EJECUTAR_COMPRA"
        
        # Ruptura de media con fuerza masiva (Eficiencia > 20%)
        if estado["fase_ciclo"] == "m" and eficiencia > 20.0:
            print(f"🚀 TENDENCIA VALIDADA | Eficiencia: {eficiencia:.2f}%")
            return "EJECUTAR_COMPRA_TREND"

    # --- ESCENARIO B: GESTIÓN DE SALIDA (INTELIGENCIA) ---
    else:
        # Salida por debilidad: Si el CVD se vuelve negativo o la eficiencia cae a casi 0
        # indicando que ya no hay presión compradora.
        if estado["fase_ciclo"] in ["sup1", "m"] and eficiencia < 2.0:
            print(f"🧊 SALIDA POR AGOTAMIENTO | Eficiencia: {eficiencia:.2f}%")
            return "EJECUTAR_VENTA"
            
    return "MANTENER"